# Random Voting Games & Quantum Shapley Values
This notebook generates random voting games and uses a quantum algorithm to estimate the Shapley value of each player. 
It then performs some basic data analysis on the predictions.

### Import Libraries

In [ ]:
import quantumBasicVotingGame as vg
from quantumShapEstimation import QuantumShapleyWrapper as qsw

import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm.auto import tqdm

### Define Variables

In [ ]:
numTrials = 128

minEll = 1
maxEll = 11

#Defining the different conditions
# numPlayersCond    = [4,8,12]
numPlayersCond    = [2,4,8]
thresholdBitCond  = [3,4,5]
roughVarianceCond = [1,1,2]


### Run Simulations

In [ ]:
simulations = {}

for trialNum in tqdm(range(numTrials), desc="Current Trial"):
    for ell in tqdm(range(1,maxEll), desc="Current Ell"):
        for n, thresholdBits, roughVariance in zip(
            numPlayersCond, thresholdBitCond, roughVarianceCond
        ):
            trial = (n,ell,trialNum)


            #New random game
            threshold = 2**(thresholdBits-1)

            playerVals = vg.randomVotingGame(
                numPlayers=n,
                thresholdBits=thresholdBits,
                roughVariance=roughVariance
            )

            #quantum Shapley
            qshaps = vg.quantumVotingShap(
                threshold=threshold,
                playerVals=playerVals,
                ell=ell
            )

            #classical Shapley
            cshaps = vg.classicalVotingShap(
                threshold=threshold,
                playerVals=playerVals,
            )

            #Store outcome
            simulations[trial] = (qshaps, cshaps)



### Save Results

In [ ]:
with open('shapleyVoteResults.pkl', 'wb') as f:
    pickle.dump(simulations, f)

In [ ]:
# with open('shapleyVoteResults_Apr30_11-55PM.pkl', 'rb') as fp:
#     simulations = pickle.load(fp)

### Analyze Trials 

In [ ]:
def meanAbsError(qshaps, cshaps):
    err = 0
    for qshap, cshap in zip(qshaps, cshaps):
        err += abs(qshap-cshap)
    return err

In [ ]:
plt.rcParams['figure.figsize'] = [12, 5]
fig, ax = plt.subplots(1, len(numPlayersCond))

#We're looking to find reciprocal mean abs error per trial
#For each trial with n players
for i, n in enumerate(numPlayersCond):
    #Orient data
    resultsX = []
    resultsY = []
    resultErr = []
    for ell in range(2, maxEll):
        trialOutcomes = []

        for trialNum in range(numTrials):
            qshaps, cshaps = simulations[(n,ell,trialNum)]
            trialOutcomes.append(
                meanAbsError(qshaps, cshaps)
            )
        
        trialOutcomes = np.array(trialOutcomes)
        resultsX.append(ell)
        resultsY.append(trialOutcomes.mean())
        resultErr.append(trialOutcomes.std())

        # resultsX += len(trialOutcomes) * [ell]
        # resultsY += trialOutcomes
    
    ax[i].set_title(f"{n} Players")#, Threshold: {2**thresholdBitCond[i]}")
    ax[i].bar(
        np.array(resultsX), 
        1/np.array(resultsY),
        # yerr=resultErr,
        align='center',
        alpha=0.5,
        ecolor='black',
        capsize=10,
    )
    ax[i].set_xlabel(r"$\ell$")
    ax[i].set_ylabel(r"Reciprocal Mean Absolute Error")
    print(f"{n=}:", 1/np.array(resultsY))

plt.tight_layout()
plt.show()